In [1]:
import numpy as np
from scipy.integrate import solve_ivp

In [83]:
import h5py

def read_drived_evolution(file_name, freq_num):
    with h5py.File(file_name, "r") as f:
        group = f[str(freq_num)]
        p0 = group["p0"][:].flatten()
        p1 = group["p1"][:].flatten()
        s_re = group["s_re"][:].flatten()
        s_im = group["s_im"][:].flatten()
        t = group["t"][:].flatten()

    n = len(t.tolist())
       
    ρ = [np.zeros((2, 2), dtype=complex) for _ in range(n)]
    
    for i in range(n):
        ρ[i] = np.array([[p0[i], s_re[i] + 1j * s_im[i]],
                            [s_re[i] - 1j * s_im[i], p1[i]]])
    
    return t, ρ

In [84]:
freq_num = 25

tₑₓ0, ρₑₓ0 = read_drived_evolution("DATA/State_B0_SIN_DRIVE_data.h5", str(freq_num))
tₑₓ1, ρₑₓ1 = read_drived_evolution("DATA/State_B1_SIN_DRIVE_data.h5", str(freq_num))
tₑₓx, ρₑₓx = read_drived_evolution("DATA/State_BX_SIN_DRIVE_data.h5", str(freq_num))
tₑₓy, ρₑₓy = read_drived_evolution("DATA/State_BY_SIN_DRIVE_data.h5", str(freq_num))

In [85]:
def bloch_vector(ρ):
    """
    Calculate the Bloch vector from a 2x2 density matrix.
    """
    σ_x = np.array([[0, 1], [1, 0]])  # Pauli-X
    σ_y = np.array([[0, -1j], [1j, 0]])  # Pauli-Y
    σ_z = np.array([[1, 0], [0, -1]])  # Pauli-Z
    
    b_x = np.trace(ρ @ σ_x).real
    b_y = np.trace(ρ @ σ_y).real
    b_z = np.trace(ρ @ σ_z).real
    
    return np.array([b_x, b_y, b_z])

def compute_bloch_vectors(ρ_solution):
    """
    Compute Bloch vectors for all density matrices in the solution.
    """
    return np.array([bloch_vector(ρ) for ρ in ρ_solution])


def simulate_bloch(H0, V, J, ρ0, tmax, dt, freq_num):
    qubit_freq = 25.133
    w = 2 * qubit_freq * (freq_num - 1) / 50
    
    def L(ρ):
        return J @ ρ @ J.conj().T - 0.5 * (J.conj().T @ J @ ρ + ρ @ J.conj().T @ J)
    
    def Ht(t):
        return H0 + V * np.sin(w * t)
    
    def matrix_ode(t, ρ_flat):
        ρ = ρ_flat.reshape((2, 2))
        dρ = -1j * (Ht(t) @ ρ - ρ @ Ht(t)) + L(ρ)
        return dρ.flatten()
    
    t_span = (0, tmax)
    t_eval = np.arange(0, tmax, dt)
    
    sol = solve_ivp(matrix_ode, t_span, ρ0.flatten(), t_eval=t_eval, method='RK45')
    ρ_solution = sol.y.T.reshape(-1, 2, 2)
    
    return compute_bloch_vectors(ρ_solution), sol.t

In [88]:
H0 = np.array([[25.133, 0.0],
               [0.0, -25.133]]) / 2

V0 = 1.0
V = V0 * np.array([[0, 1],
                   [1, 0]])

J = np.array([[0, np.sqrt(0.25133)],
              [0, 0]])

In [89]:
dt = tₑₓ0[2]-tₑₓ0[1]

b0, t0 = simulate_bloch(H0, V, J, ρₑₓ0[0], tₑₓ0[-1], dt, freq_num-1)
b1, t1 = simulate_bloch(H0, V, J, ρₑₓ1[0], tₑₓ1[-1], dt, freq_num-1)
bx, tx = simulate_bloch(H0, V, J, ρₑₓx[0], tₑₓx[-1], dt, freq_num-1)
by, ty = simulate_bloch(H0, V, J, ρₑₓy[0], tₑₓy[-1], dt, freq_num-1)

In [94]:
def stack_bloch_vectors(b0, b1, bx, by):
    """
    Stack four N x 3 Bloch vector arrays into an N x 12 array.
    Truncate arrays to the smallest N if they have different lengths.
    
    Parameters:
        b0, b1, bx, by (np.ndarray): N x 3 arrays of Bloch vectors.
    
    Returns:
        np.ndarray: N x 12 stacked Bloch vector array.
    """
    # Determine the smallest N
    N = min(len(b0), len(b1), len(bx), len(by))
    
    # Truncate all arrays to the smallest N
    b0_truncated = b0[:N]
    b1_truncated = b1[:N]
    bx_truncated = bx[:N]
    by_truncated = by[:N]
    
    # Stack arrays horizontally (axis=1)
    bloch_stack = np.hstack([b0_truncated, b1_truncated, bx_truncated, by_truncated])
    
    return bloch_stack

In [105]:
Y = stack_bloch_vectors(b0, b1, bx, by)
Y.shape[0]


1367

In [113]:
qubit_freq = 25.133
w = 2 * qubit_freq * (freq_num - 1) / 50
V0 = 1
tspan = np.arange(t0[0], t0[0] + (Y.shape[0]-1) * dt, dt)
U = np.array([V0 * np.sin(w * t) for t in tspan])


In [115]:
Y1 = Y[:-1].T  # All but the last time step (transpose for column alignment)
Y2 = Y[1:].T   # All but the first time step (transpose for column alignment)

model = biDMD(Y2, Y1, U, tspan)

In [117]:
model.A.shape

(12, 12)

In [97]:
from dmdlab import biDMD
import numpy as np

def apply_biDMD(data, n_dynamics=3, truncate_rank=5):
    """
    Apply bi-DMD to the Bloch vector stack.

    Parameters:
        data (np.ndarray): N x 12 matrix of Bloch vectors.
        n_dynamics (int): Number of dynamic modes to compute.
        truncate_rank (int): Rank truncation for SVD.

    Returns:
        dmd: Fitted biDMD object.
    """
    # Initialize bi-DMD object
    dmd = biDMD(n_dynamics=n_dynamics, truncate_rank=truncate_rank)
    
    # Fit the model to the data
    dmd.fit(data)
    
    return dmd

In [98]:
dmd_model = apply_biDMD(bloch_stack)

TypeError: biDMD.__init__() missing 4 required positional arguments: 'X2', 'X1', 'U', and 'ts'